In [52]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

#1. Price Data

# Read stock tickers from the CSV file
#tickers_df = pd.read_csv('\Resources\stock_tickers.csv', header=None)
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'IBM', 'NVDA', 'JPM', 'V', 'UNH']
tickers_df = pd.DataFrame(tickers)
Nasdaq_NYSE_10 = tickers_df[0].tolist()

st.title("Stock Ticker Data Analysis")

# User input using Streamlit components
#selected_ticker = st.selectbox("Select a stock ticker from the list:", Nasdaq_NYSE_10)
selected_ticker = "AAPL"
start_date = st.date_input("Start Date:", datetime.now() - timedelta(days=365))
end_date = st.date_input("End Date:")

# Fetch historical price data using yfinance
stock_data = yf.download(selected_ticker, start=start_date_str, end=end_date_str)

# Create a DataFrame with selected price data
stock_df = pd.DataFrame()
stock_df['High'] = stock_data['High']
stock_df['Low'] = stock_data['Low']
stock_df['Open'] = stock_data['Open']
stock_df['Close'] = stock_data['Close']

print(stock_df.head())

# ----------------------------------------------------------------

    # Functions for calculations
def calculate_ema(stock_df, window):
    return stock_df['Close'].ewm(span=window, adjust=False).mean()

def backtest_crossover_strategy(stock_df, short_window, long_window):
    short_ema = calculate_ema(stock_df, short_window)
    long_ema = calculate_ema(stock_df, long_window)

    # Buy signal: when short EMA crosses above long EMA
    stock_df['Buy Signal'] = (short_ema > long_ema) & (short_ema.shift(1) <= long_ema.shift(1))

    # Sell signal: when short EMA crosses below long EMA
    stock_df['Sell Signal'] = (short_ema < long_ema) & (short_ema.shift(1) >= long_ema.shift(1))

    # Assume starting with cash, so the first signal should be a buy
    initial_position = "cash"
    positions = [initial_position]

    for i in range(1, len(stock_df)):
        if stock_df['Buy Signal'].iloc[i]:
            positions.append("stock")
        elif stock_df['Sell Signal'].iloc[i]:
            positions.append("cash")
        else:
            positions.append(positions[-1])

    stock_df['Position'] = positions
    
    # Calculate returns
    stock_df['Daily Strategy Return'] = np.where(stock_df['Position'] == "stock", stock_df['Close'].pct_change(), 0)
    total_strategy_return = (stock_df['Daily Strategy Return'] + 1).prod()
    return total_strategy_return
    

# test strategy    
short_window = 40  # or any value you'd like to use
long_window = 100  # or any value you'd like to use

total_strategy_return = backtest_crossover_strategy(stock_df, short_window, long_window)
stock_df.to_csv('ema_crossover.csv', index=False)
total_strategy_return = 100*(total_strategy_return-1)
print(f"Total return of EMA Crossover Strategy: {total_strategy_return:.2f}%")

[*********************100%***********************]  1 of 1 completed
                  High         Low        Open       Close
Date                                                      
2022-08-22  169.860001  167.139999  169.690002  167.570007
2022-08-23  168.710007  166.649994  167.080002  167.229996
2022-08-24  168.110001  166.250000  167.320007  167.529999
2022-08-25  170.139999  168.350006  168.779999  170.029999
2022-08-26  171.050003  163.559998  170.570007  163.619995
Total return of EMA Crossover Strategy: 22.31%
